#### FAISS(Facebook AI Similarity Search)

A library that allows us to quickly search for documnets that are similar to each other. It also contains supporting code for evaluation and parameter tuning 

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings


In [5]:
loader = TextLoader("speech.txt")
document = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=10)
docs = text_splitter.split_documents(document)

docs

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [7]:
embeddings = OllamaEmbeddings(model="llama3.2:1b", base_url="http://host.docker.internal:11434")
db = FAISS.from_documents(docs, embeddings)
db

In [8]:
## Give Input query
input_query = "How does the speakler describe the desired outcome of the war?"

docs = db.similarity_search(input_query)
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

#### As a Retriever

Acts as a interface usingb which we will be able to retrieve the information from the vector store and provide the response.

In [9]:
retriever = db.as_retriever()
docs = retriever.invoke(input_query)
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

#### Similarity search with score

Returns not only the document but also the distance score of the query to them, Distance score is L2 distnace. LOwer score is better.

In [10]:
docs_and_score = db.similarity_search_with_score(input_query)

docs_and_score

[(Document(id='6959da0c-db2b-464b-bea2-cccde6e51aaa', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  0.4578994),
 (Document(id='9b582ba8-5b7a-4e16-888a-14c81e9c38ea', metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friend

In [11]:
## Provide vectors instead of documents

embed_vector = embeddings.embed_query(input_query)

embed_vector

[-0.014999371,
 0.019995296,
 0.016220853,
 -0.010660628,
 0.058495056,
 -0.040771205,
 -0.0032869428,
 0.033739846,
 -0.0064083296,
 0.008599426,
 -0.007926198,
 -0.027331414,
 -0.0047321212,
 -0.000926721,
 0.037141483,
 -0.016884984,
 0.001109708,
 -0.00089809636,
 0.08424737,
 0.008981771,
 -0.011490492,
 0.010950328,
 -0.009794129,
 -0.0057070875,
 -0.01746985,
 0.01249751,
 -0.025063118,
 0.03063192,
 -0.0013815954,
 0.01773715,
 -0.005682401,
 0.0154902665,
 -0.021818556,
 0.022676386,
 -0.011062148,
 -0.005591262,
 -0.00790556,
 -0.010498562,
 7.1870556e-05,
 -0.009431147,
 -0.007028193,
 -0.013123579,
 -0.008720256,
 0.03310847,
 -0.0025488953,
 0.00889394,
 0.022571702,
 -0.019922938,
 0.031710282,
 -0.023308942,
 -0.0025422496,
 -0.006953126,
 0.050951492,
 -0.004180862,
 -0.02658782,
 -0.0090223765,
 -0.020289917,
 0.013414319,
 0.010742549,
 0.020335373,
 0.005451603,
 0.0027843313,
 -0.027820671,
 0.0060529923,
 0.0097583085,
 -0.007063807,
 -0.0145856915,
 -0.0011551466,

In [ ]:
docs_and_score = db.similarity_search_by_vector(embed_vector)
docs_and_score

[Document(id='6959da0c-db2b-464b-bea2-cccde6e51aaa', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='9b582ba8-5b7a-4e16-888a-14c81e9c38ea', metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German 

In [13]:
## Save the vector db to local and loading

db.save_local("Faiss_index")

In [17]:
new_db = FAISS.load_local("Faiss_index", embeddings, allow_dangerous_deserialization=True)

docs = db.similarity_search(input_query)
docs

[Document(id='6959da0c-db2b-464b-bea2-cccde6e51aaa', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='9b582ba8-5b7a-4e16-888a-14c81e9c38ea', metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German 